In [ ]:
!pip install wikipedia~=1.4 duckduckgo-search==6.0.0 lxml==5.3.0

In [1]:
from bedrock_agent.utils import get_boto_session
from bedrock_agent.tool import tool
from bedrock_agent.types import ModelConfig
from bedrock_agent.agent import BedrockAgent
from dotenv import load_dotenv
import json

import wikipedia
from duckduckgo_search import DDGS, exceptions

load_dotenv()

session = get_boto_session()

In [2]:
@tool
def search_wikipedia(search_term: str, lang: str = "en", max_results: int = 3):
    """ 
    Searches Wikipedia for the given search term and returns summaries of the results.
    
    :param search_term: The term to search for on Wikipedia.
    :param lang: The language in which to search. Defaults to "en".
    :param max_results: The maximum number of search results to return. Defaults to 3.
    
    :returns: A list of page summaries for the search results. If no results are found, returns "No search results." If a page cannot be loaded, returns "Unable to load page."
    """
    wikipedia.set_lang(lang)
    
    pages = wikipedia.search(search_term, results=max_results)
    if len(pages) == 0:
        return "No search results."
    
    page_summaries = []
    for page in pages:
        try:
            page_summary = wikipedia.summary(page, auto_suggest=False)
            page_summaries.append(page_summary)
        except wikipedia.exceptions.PageError:
            page_summaries.append("Unable to load page.")
    return page_summaries

@tool
def fetch_duckduckgo_results(query: str, region: str = "de-de", max_results: int = 5):
    """ 
    Searches DuckDuckGo (Web-Search) for the given query and returns a list of the results.
    
    :param query: The query to search for on DuckDuckGo.
    :param region: The region in which to search. Defaults to "de-de".
    :param max_results: The maximum number of search results to return. Defaults to 5.
    
    :returns: A list of search results. If the search fails, returns "Ratelimited. Try again later."
    """
    params = {
        "keywords": query,
        "region": region,
        "max_results": max_results,
        "backend": "html",
    }
    headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/122.0.6261.95 Safari/537.36"
        }
    with DDGS(headers=headers) as ddg:
        for _ in range(3):
            try:
                results = list(ddg.text(**params))
                return json.dumps([res.get("body") for res in results])
            except exceptions.RatelimitException:
                return "Ratelimited. Try again later."

In [3]:
tools = [search_wikipedia, fetch_duckduckgo_results]
model_cfg = ModelConfig(model_id="amazon.nova-pro-v1:0")
agent = BedrockAgent("ExpertAgent", session, model_cfg=model_cfg, tools=tools, verbose=False)

In [ ]:
agent.chat("Can you please analyze the latest iPhone 16 Pro? I need a summary of the features and specifications. Please think step by step and also try to browse the web if you need additional information.")

In [5]:
agent.draw_trace()


--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: Can you please analyze the latest iPhone 16 Pro? I need a summary of the features and specifications. Please think step by step and also try to browse the web if you need additional information.

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TOOL USE: search_wikipedia
INPUT: {'search_term': 'iPhone 16 Pro', 'max_results': 1}

--------------------------------------------------
Message # 3
--------------------------------------------------
ROLE: USER
TOOL RESULT: ['The iPhone 16 Pro and iPhone 16 Pro Max are smartphones developed and marketed by Apple Inc. Alongside the iPhone 16 and iPhone 16 Plus, they form the eighteenth generation of the iPhone, succeeding the iPhone 15 Pro and iPhone 15 Pro Max, and were announced on September 9, 2024, and released on September 20, 2024.

In [ ]:
agent.chat("Can you please fetch the latest news about the iPhone 16 Pro and tell me if its a good investment?")

In [7]:
agent.draw_trace()


--------------------------------------------------
Message # 1
--------------------------------------------------
ROLE: USER
TEXT: Can you please analyze the latest iPhone 16 Pro? I need a summary of the features and specifications. Please think step by step and also try to browse the web if you need additional information.

--------------------------------------------------
Message # 2
--------------------------------------------------
ROLE: ASSISTANT
TOOL USE: search_wikipedia
INPUT: {'search_term': 'iPhone 16 Pro', 'max_results': 1}

--------------------------------------------------
Message # 3
--------------------------------------------------
ROLE: USER
TOOL RESULT: ['The iPhone 16 Pro and iPhone 16 Pro Max are smartphones developed and marketed by Apple Inc. Alongside the iPhone 16 and iPhone 16 Plus, they form the eighteenth generation of the iPhone, succeeding the iPhone 15 Pro and iPhone 15 Pro Max, and were announced on September 9, 2024, and released on September 20, 2024.

In [8]:
print(f"Total tokens used: {agent.total_tokens_used} | total costs: {agent.total_costs} USD")

Total tokens used: 3402 | total costs: 0.0042288000 USD
